#Translation Using Word Level Seq2Seq

In [1]:
%tensorflow_version 1.14
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
from tensorflow import keras
from IPython import display
from matplotlib import cm
from matplotlib import gridspec
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from tensorflow.python.data import Dataset
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
print(tf.__version__)
import string
from string import digits
%matplotlib inline
import re
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from keras.layers import Input, LSTM, Embedding, Dense
from keras.models import Model


`%tensorflow_version` only switches the major version: 1.x or 2.x.
You set: `1.14`. This will be interpreted as: `1.x`.


TensorFlow 1.x selected.
Instructions for updating:
non-resource variables are not supported in the long term
1.15.2


Using TensorFlow backend.


In [0]:
lines= pd.read_table('hin.txt', names=['eng','hin','otherdata'])

In [0]:
lines = lines[0:50000]

In [5]:
lines.shape

(2778, 3)

In [6]:
lines.sample(10)

,eng,hin,otherdata
1005,He was sentenced to death.,उसे मौत की सज़ा दे दी गई।,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
2195,Workers must have their hair cut short.,कर्मचारियों को अपने बाल छोटे कटाने होते हैं।,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
150,He has a beard.,उसके पास दाढ़ी है।,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
1417,He took advantage of my youth.,उसने मेरे जवान होने का फ़ायदा उठाया।,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
1290,Tom isn't able to drive yet.,टॉम अभी गाड़ी नहीं चला सकता।,CC-BY 2.0 (France) Attribution: tatoeba.org #1...
798,He couldn't get the job.,उसको नौकरी नहीं मिल पायी।,CC-BY 2.0 (France) Attribution: tatoeba.org #4...
1572,Why were you late this morning?,तुम आज सुबह देर से क्यों आए?,CC-BY 2.0 (France) Attribution: tatoeba.org #3...
1516,I have a bone to pick with you.,मैं तुमसे खुश नहीं हूँ।,CC-BY 2.0 (France) Attribution: tatoeba.org #1...
517,He envied my success.,उसको मेरी कामयाबी से जलन थी।,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
1898,Learning a foreign language is fun.,विदेषी भाषा सीखना बहुत मज़ेदार होता है।,CC-BY 2.0 (France) Attribution: tatoeba.org #2...


In [0]:
lines.eng=lines.eng.apply(lambda x: x.lower())
lines.hin=lines.hin.apply(lambda x: x.lower())

In [0]:
# Take the length as 50
lines.eng=lines.eng.apply(lambda x: re.sub("'", '', x)).apply(lambda x: re.sub(",", ' COMMA', x))
lines.hin=lines.hin.apply(lambda x: re.sub("'", '', x)).apply(lambda x: re.sub(",", ' COMMA', x))

In [0]:
exclude = set(string.punctuation)
lines.eng=lines.eng.apply(lambda x: ''.join(ch for ch in x if ch not in exclude))
lines.hin=lines.hin.apply(lambda x: ''.join(ch for ch in x if ch not in exclude))

In [0]:
remove_digits = str.maketrans('', '', digits)
lines.eng=lines.eng.apply(lambda x: x.translate(remove_digits))
lines.hin=lines.hin.apply(lambda x: x.translate(remove_digits))

In [11]:
lines.head()

,eng,hin,otherdata
0,wow,वाह,CC-BY 2.0 (France) Attribution: tatoeba.org #5...
1,help,बचाओ,CC-BY 2.0 (France) Attribution: tatoeba.org #4...
2,jump,उछलो,CC-BY 2.0 (France) Attribution: tatoeba.org #6...
3,jump,कूदो,CC-BY 2.0 (France) Attribution: tatoeba.org #6...
4,jump,छलांग,CC-BY 2.0 (France) Attribution: tatoeba.org #6...


In [0]:
lines.hin = lines.hin.apply(lambda x : 'START_ '+ x + ' _END')

In [13]:
lines.hin.head()

0      START_ वाह _END
1     START_ बचाओ _END
2     START_ उछलो _END
3     START_ कूदो _END
4    START_ छलांग _END
Name: hin, dtype: object

In [0]:
all_eng_words=set()
for eng in lines.eng:
    for word in eng.split():
        if word not in all_eng_words:
            all_eng_words.add(word)
    
all_hindi_words=set()
for hin in lines.hin:
    for word in hin.split():
        if word not in all_hindi_words:
            all_hindi_words.add(word)


In [15]:
len(all_eng_words), len(all_hindi_words)

(2345, 2991)

In [16]:
lenght_list=[]
for l in lines.hin:
    lenght_list.append(len(l.split(' ')))
np.max(lenght_list)

29

In [17]:
lenght_list=[]
for l in lines.eng:
    lenght_list.append(len(l.split(' ')))
np.max(lenght_list)

23

In [0]:
input_words = sorted(list(all_eng_words))
target_words = sorted(list(all_hindi_words))
num_encoder_tokens = len(all_eng_words)
num_decoder_tokens = len(all_hindi_words)

In [0]:
input_token_index = dict(
    [(word, i) for i, word in enumerate(input_words)])
target_token_index = dict(
    [(word, i) for i, word in enumerate(target_words)])

In [0]:
encoder_input_data = np.zeros(
    (len(lines.eng), 23),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(lines.hin), 29),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(lines.hin), 29, num_decoder_tokens),
    dtype='float32')

In [0]:
for i, (input_text, target_text) in enumerate(zip(lines.eng, lines.hin)):
    for t, word in enumerate(input_text.split()):
        encoder_input_data[i, t] = input_token_index[word]
    for t, word in enumerate(target_text.split()):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t] = target_token_index[word]
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[word]] = 1.


###Build keras encoder-decoder model

In [0]:
embedding_size = 100

In [0]:
from keras.layers import Input, LSTM, Embedding, Dense
from keras.models import Model
from keras.utils import plot_model

In [26]:
#Encoder model
encoder_inputs = Input(shape=(None,))
en_x=  Embedding(num_encoder_tokens, embedding_size)(encoder_inputs)
encoder = LSTM(50, return_state=True)
encoder_outputs, state_h, state_c = encoder(en_x)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

In [27]:
#Decoder model
# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None,))

dex=  Embedding(num_decoder_tokens, embedding_size)

final_dex= dex(decoder_inputs)


decoder_lstm = LSTM(50, return_sequences=True, return_state=True)

decoder_outputs, _, _ = decoder_lstm(final_dex,
                                     initial_state=encoder_states)

decoder_dense = Dense(num_decoder_tokens, activation='softmax')

decoder_outputs = decoder_dense(decoder_outputs)

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])

In [28]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, None, 100)    234500      input_1[0][0]                    
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, None, 100)    299100      input_2[0][0]                    
____________________________________________________________________________________________

In [29]:
#Fit the model
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=128,
          epochs=500,
          validation_split=0.05)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Train on 2639 samples, validate on 139 samples
Epoch 1/500





2639/2639 [==============================] - 11s 4ms/step - loss: 1.9295 - acc: 0.0321 - val_loss: 3.1524 - val_acc: 0.0345
Epoch 2/500
2639/2639 [==============================] - 2s 694us/step - loss: 1.6307 - acc: 0.0345 - val_loss: 3.0058 - val_acc: 0.0345
Epoch 3/500
2639/2639 [==============================] - 2s 693us/step - loss: 1.5500 - acc: 0.0345 - val_loss: 3.0096 - val_acc: 0.0345
Epoch 4/500
2639/2639 [==============================] - 2s 681us/step - loss: 1.5211 - acc: 0.0345 - val_loss: 3.0187 - val_acc: 0.0345
Epoch 5/500
2639/2639 [==============================] - 2s 677us/step - loss: 1.4997 - acc: 0.0345 - val_loss: 3.0235 - val_acc: 0.0345
Epoch 6/500
2639/2639 [==============================] - 2s 674us/step - loss: 1.4794 - acc: 0.0345 - val_loss: 3.0146 - val_acc: 0.0345
Epoch 7/500
2639/2639 [=======

In [0]:
encoder_model = Model(encoder_inputs, encoder_states)
encoder_model.summary()

In [0]:
#Create sampling model
decoder_state_input_h = Input(shape=(50,))
decoder_state_input_c = Input(shape=(50,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

final_dex2= dex(decoder_inputs)

decoder_outputs2, state_h2, state_c2 = decoder_lstm(final_dex2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]
decoder_outputs2 = decoder_dense(decoder_outputs2)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2)

# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())

In [0]:
#Function to generate sequences
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0] = target_token_index['START_']

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += ' '+sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '_END' or
           len(decoded_sentence) > 52):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [33]:
#Look at some translations
for seq_index in range(100):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', lines.eng[seq_index: seq_index + 1])
    print('Decoded sentence:', decoded_sentence)

-
Input sentence: 0    wow
Name: eng, dtype: object
Decoded sentence:  वाह _END
-
Input sentence: 1    help
Name: eng, dtype: object
Decoded sentence:  कुरसियाँ काफ़ी हैं क्या _END
-
Input sentence: 2    jump
Name: eng, dtype: object
Decoded sentence:  कूदो _END
-
Input sentence: 3    jump
Name: eng, dtype: object
Decoded sentence:  कूदो _END
-
Input sentence: 4    jump
Name: eng, dtype: object
Decoded sentence:  कूदो _END
-
Input sentence: 5    hello
Name: eng, dtype: object
Decoded sentence:  नमस्कार। _END
-
Input sentence: 6    hello
Name: eng, dtype: object
Decoded sentence:  नमस्कार। _END
-
Input sentence: 7    cheers
Name: eng, dtype: object
Decoded sentence:  वाहवाह _END
-
Input sentence: 8    cheers
Name: eng, dtype: object
Decoded sentence:  वाहवाह _END
-
Input sentence: 9    got it
Name: eng, dtype: object
Decoded sentence:  समझे कि नहीं _END
-
Input sentence: 10    im ok
Name: eng, dtype: object
Decoded sentence:  मैं ठीक हूँ। _END
-
Input sentence: 11    awesome
Name: eng, 

#Summary:
### From the above, we can say that most of the sentences are being translated correctly.